In [1]:
file_path = '/content/drive/My Drive/movie_genre/train_data.txt'

In [2]:
import pandas as pd
df = pd.read_csv(file_path, sep=':::', header=None, names=['ID', 'Title', 'Genre', 'Description'])

<ipython-input-2-fa36d8f51a96>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep=':::', header=None, names=['ID', 'Title', 'Genre', 'Description'])


In [44]:
!pip install -U accelerate
!pip install -U transformers

  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (9

In [3]:
df['Genre'] = df['Genre'].str.split(',')

In [4]:
genre_counts = [g for gen in df['Genre'] for g in gen]
pd.Series(genre_counts).value_counts()

 drama           13613
 documentary     13096
 comedy           7447
 short            5073
 horror           2204
 thriller         1591
 action           1315
 western          1032
 reality-tv        884
 family            784
 adventure         775
 music             731
 romance           672
 sci-fi            647
 adult             590
 crime             505
 animation         498
 sport             432
 talk-show         391
 fantasy           323
 mystery           319
 musical           277
 biography         265
 history           243
 game-show         194
 news              181
 war               132
Name: count, dtype: int64

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel = MultiLabelBinarizer()

labels = multilabel.fit_transform(df['Genre']).astype('float32')

texts = df['Description'].tolist()

In [6]:
labels
texts[:5]

[' Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.',
 ' A brother and sister with a past incestuous relationship have a current murderous relationship. He murders the women who reject him and she murders the women who get too close to him.',
 ' As the bus empties the students for their field trip to the Museum of Natural History, little does the tour guide suspect that the students are there for more than just another tour. First, during the lecture films, the coeds drift into dreams of the most e

In [7]:
import torch
from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [8]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels,
                                                                    test_size=0.2, random_state=42)

In [9]:
checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels[0]),
                                                            problem_type="multi_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
labels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [11]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=128):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len, return_tensors='pt')

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': label
    }

In [12]:
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

In [41]:
tokenizer.save_pretrained("distilbert-finetuned-MovieGenre-model")

('distilbert-finetuned-MovieGenre-model/tokenizer_config.json',
 'distilbert-finetuned-MovieGenre-model/special_tokens_map.json',
 'distilbert-finetuned-MovieGenre-model/vocab.txt',
 'distilbert-finetuned-MovieGenre-model/added_tokens.json')

In [13]:
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction
import torch


def multi_labels_metrics(predictions, labels, threshold=0.3):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs>=threshold)] = 1
  y_true = labels

  f1 = f1_score(y_true, y_pred, average = 'macro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'macro')
  hamming = hamming_loss(y_true, y_pred)

  metrics = {
      "roc_auc": roc_auc,
      "hamming_loss": hamming,
      "f1": f1
  }

  return metrics

def compute_metrics(p:EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

  result = multi_labels_metrics(predictions=preds,
                                labels=p.label_ids)

  return result

In [14]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir = './results',
    num_train_epochs=2,
    save_steps=1000,
    save_total_limit=2
)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_dataset,
                  eval_dataset = val_dataset,
                  compute_metrics=compute_metrics)

In [15]:
trainer.train()

Step,Training Loss
500,0.134800
1000,0.090400
1500,0.084200
2000,0.079800
2500,0.076500
3000,0.078300
3500,0.073000
4000,0.072600
4500,0.071000
5000,0.069900


TrainOutput(global_step=10844, training_loss=0.06708845517064584, metrics={'train_runtime': 1350.9386, 'train_samples_per_second': 64.209, 'train_steps_per_second': 8.027, 'total_flos': 2873902501836288.0, 'train_loss': 0.06708845517064584, 'epoch': 2.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.066947340965271,
 'eval_roc_auc': 0.6706694138454834,
 'eval_hamming_loss': 0.026509678543248587,
 'eval_f1': 0.36838714227685415,
 'eval_runtime': 70.3476,
 'eval_samples_per_second': 154.135,
 'eval_steps_per_second': 19.276,
 'epoch': 1.0}

In [33]:
trainer.save_model("distilbert-finetuned-MovieGenre-model")

In [6]:
import pickle
with open("multi-label-binarizer.pkl", "wb") as f:
  pickle.dump(multilabel, f)

NameError: name 'multilabel' is not defined

In [2]:


text = """After a year traveling the world, twentysomething Ciaran reluctantly returns to his hometown of Dublin, broke and stuck living with his parents. As he struggles to reconnect with the ex-girlfriend that he left behind and the friends and social scene that have moved on without him, Ciaran leads us through an atmospheric and authentic vision of contemporary Dublin and Irish youth.

"""

encoding = tokenizer(text, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)

sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu())
preds = np.zeros(probs.shape)
preds[np.where(probs>=0.3)] = 1

multilabel.classes_

multilabel.inverse_transform(preds.reshape(1,-1))

NameError: name 'tokenizer' is not defined

In [1]:
preds

NameError: name 'preds' is not defined

In [16]:
import pickle
with open("multi-label-binarizer.pkl", "rb") as f:
  loaded_model = pickle.load(f)

EOFError: Ran out of input

In [16]:
# Save the model and tokenizer in a directory
model.save_pretrained('distilbert-finetuned-MovieGenre-model')
tokenizer.save_pretrained('distilbert-finetuned-MovieGenre-model')


('distilbert-finetuned-MovieGenre-model/tokenizer_config.json',
 'distilbert-finetuned-MovieGenre-model/special_tokens_map.json',
 'distilbert-finetuned-MovieGenre-model/vocab.txt',
 'distilbert-finetuned-MovieGenre-model/added_tokens.json')

In [17]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Create a path in Google Drive
drive_path = '/content/drive/My Drive/distilbert-finetuned-MovieGenre-model'

# Copy the directory to Google Drive
shutil.copytree('distilbert-finetuned-MovieGenre-model', drive_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/distilbert-finetuned-MovieGenre-model'